In [55]:
!pip install ccxt
!pip install pandas_ta

In [56]:
import ccxt
from datetime import datetime as datetime,timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as ta
import warnings
import time
import pytz
import math

warnings.filterwarnings('ignore')

apiKey = ''
secretKey = ''
# !pip install -U vectorbt
ex = ccxt.binance({'apiKey':y,'secret':z,'enableRateLimit':True})

In [57]:
ticker_list = ['BTC/BUSD','ETH/BUSD','BNB/BUSD']

value_ = [10,10,10] # $ Usd value /symbols


bot_setting= dict(zip(ticker_list,value_))
bot_setting

{'BTC/BUSD': 10, 'ETH/BUSD': 10, 'BNB/BUSD': 10}

In [58]:
def get_data(sym,timeframe='1m',limit=500):
  bars = ex.fetch_ohlcv(sym, timeframe=timeframe, limit=limit)
  df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
  df = df.set_index('timestamp')
  return df


def download_data(ticker_list,timeframe):
  coin_dict = {}
  for ticker in ticker_list:
    df= get_data(ticker,timeframe=timeframe)
    coin_dict[ticker] = df
  df = pd.concat(coin_dict,axis=0)
  return df

#
def strategy(df):
  df['ema'] =  df.ta.ema(length=3)
  df['sma'] = df.ta.sma(length=3)
  df['trend'] = df['ema'] > df['sma'] 
  df['signal'] =    np.where((df['trend']==True) & (df['trend'].shift()==False), 'buy', np.NaN)
  df['signal'] =    np.where((df['trend']==False) & (df['trend'].shift()==True), 'sell', df['signal'])
  return df

In [59]:
# Download Data
coin_dict = {}

for ticker in ticker_list:
  df= get_data(ticker)
  coin_dict[ticker] = df
coin_dict

{'BTC/BUSD':                          open      high       low     close     volume
 timestamp                                                             
 2023-01-13 11:23:00  18988.68  18995.26  18985.15  18992.50   80.02745
 2023-01-13 11:24:00  18992.27  18995.70  18982.33  18982.73   76.00853
 2023-01-13 11:25:00  18982.37  18997.99  18980.00  18989.99  114.52741
 2023-01-13 11:26:00  18990.05  18991.37  18977.49  18981.15  104.38165
 2023-01-13 11:27:00  18980.38  18980.88  18970.09  18976.04   74.00789
 ...                       ...       ...       ...       ...        ...
 2023-01-13 19:38:00  19295.97  19302.20  19291.71  19300.40   67.49489
 2023-01-13 19:39:00  19300.41  19313.73  19299.35  19307.10   91.75117
 2023-01-13 19:40:00  19308.07  19316.74  19302.20  19315.76   67.97206
 2023-01-13 19:41:00  19315.91  19334.00  19313.02  19324.54  158.97307
 2023-01-13 19:42:00  19324.50  19330.82  19323.31  19329.40   11.08996
 
 [500 rows x 5 columns],
 'ETH/BUSD':             

In [60]:
# Combine Data
df = pd.concat(coin_dict,axis=0)
df

open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 11:23:00  18988.68  18995.26  18985.15  18992.50   
         2023-01-13 11:24:00  18992.27  18995.70  18982.33  18982.73   
         2023-01-13 11:25:00  18982.37  18997.99  18980.00  18989.99   
         2023-01-13 11:26:00  18990.05  18991.37  18977.49  18981.15   
         2023-01-13 11:27:00  18980.38  18980.88  18970.09  18976.04   
...                                ...       ...       ...       ...   
BNB/BUSD 2023-01-13 19:38:00    287.60    287.70    287.50    287.70   
         2023-01-13 19:39:00    287.60    287.80    287.60    287.70   
         2023-01-13 19:40:00    287.80    287.80    287.70    287.80   
         2023-01-13 19:41:00    287.80    287.90    287.70    287.80   
         2023-01-13 19:42:00    287.80    287.90    287.80    287.90   

                                 volume  
         timestamp                       
BTC/BUSD 2023-01-13 11:23:00   80.02745  
         2023-01-13 11:24:00   76.00853  
         2023-01-13 11:25:00  114.52741  
         2023-01-13 11:26:00  104.38165  
         2023-01-13 11:27:00   74.00789  
...                                 ...  
BNB/BUSD 2023-01-13 19:38:00   32.25200  
         2023-01-13 19:39:00   41.90600  
         2023-01-13 19:40:00   55.67800  
         2023-01-13 19:41:00  225.17200  
         2023-01-13 19:42:00   17.29200  

[1500 rows x 5 columns]

In [61]:
def get_wallet():
    res = ex.fetch_balance()
    wallet= pd.DataFrame(res['info']['balances'])
    wallet['free'] =wallet['free'].astype(float)
    wallet =wallet.sort_values('free',ascending=False)
    return wallet


def get_time():
    time_str=datetime.now(pytz.timezone('Asia/Bangkok')).strftime('%Y-%m-%d' ' %H:%M:%S')
    return time_str

def sell_market(symbol,size_=0):
    try:
        wallet= get_wallet()
        name = symbol.split('/')[0]
        if size_== 0 :
            size_= wallet[wallet['asset']== name ]['free'].values[0] # check values in wallet 
        res=ex.create_order(symbol,'market','sell',size_) # sell all
        print(f'Sell {symbol}  Market order {size_} unit ' )
    except Exception as e :
        print(get_time(),' : ',str(e))
        res =[]
    return res

def buy_market(symbol,size_):
    try:
        res= ex.create_order(symbol,'market','buy',size_)
        print(f'Buy {symbol}   Market order {size_} unit ' )

    except Exception as e:
        print(get_time(),' : ',str(e))
        res= []
    except ccxt.InvalidOrder as e:
        last,digit,min_size=get_detail()
        size_ = size_ + min_size 
        res= ex.create_order(symbol,'market','buy',size_)
        print(f'Buy Market order {symbol} {size_} unit ' )
    return res

In [62]:
def roundup(x,increment=5):
  # step_incre,ment
  return int(math.ceil(x / increment)) * increment

def get_min_size(symbol):
    res= ex.market(symbol)
    print(res['info']['filters'])
    min_size= float(res['info']['filters'][1]['minQty'])
    return min_size

def get_digit(symbol):
    price_res = ex.fetch_ticker(symbol)
    last = price_res['last']
    min_size = get_min_size(symbol)
    digit_ =  float(min_size)+1
    digit= len(str(digit_).split('.')[1])
    # print(price_res['close'],'digit :',digit,min_size)
    return digit

In [63]:
# Test Download 
df= download_data(ticker_list,timeframe='1h')
stg = strategy(df)
signal = stg['signal'].unstack(0)
signal

,BTC/BUSD,ETH/BUSD,BNB/BUSD
timestamp,,,
2022-12-24 00:00:00,nan,nan,nan
2022-12-24 01:00:00,nan,nan,nan
2022-12-24 02:00:00,nan,buy,buy
2022-12-24 03:00:00,buy,nan,nan
2022-12-24 04:00:00,sell,nan,nan
...,...,...,...
2023-01-13 15:00:00,nan,nan,nan
2023-01-13 16:00:00,sell,sell,sell
2023-01-13 17:00:00,nan,nan,nan


In [64]:
buy_log= []
sell_log = []

df= download_data(ticker_list,'1h')
stg = strategy(df)
signal = stg['signal'].unstack(0)

for ticker in ticker_list:
    last_sig =signal[ticker][-1]
    fix_value = bot_setting[ticker]
    price = ex.fetch_ticker(ticker)['last']
    min_size= get_min_size(ticker)
    ############## AMOUNT CHECK SIZE RESIZE TO Avaiable Trades
    amount = fix_value / price
    amount_ = roundup(amount,min_size)
    amount_ = round(amount_,get_digit(ticker))
    value_ = round(amount_  * price,2)
    time_= get_time()
    if last_sig == 'buy':
        msg= f'[{time_}] Buy {ticker} {price} {amount_} {value_}  '
        print(msg)
        print('#'*50)

      # buy_res = buy_market(sym,amount_)
  
        buy_res= []
        if buy_res !=None:
        #hold pos =true
            print('BUY Complete : ')
        else:
            print(f'Buy Not Complete : {amount_}')
            print('#'*50)
    elif last_sig =='sell':
        last_sig =signal[ticker][-1]
        msg= f'[{time_}] Sell {ticker} {price} {amount_} {value_}  '
        print(msg)
        print('#'*50)

      # sell_res = sell_market_all(sym)
        sell_res= []
        if sell_res !=None:
        #hold pos =False

            print('Sell Complete : ')
        else:
            print(f'Sell Not Complete : {amount_}')
            print('#'*50)
    else:
        print(ticker,' No Signal',)
        print('#'*50)

[{'filterType': 'PRICE_FILTER', 'minPrice': '0.01000000', 'maxPrice': '1000000.00000000', 'tickSize': '0.01000000'}, {'filterType': 'LOT_SIZE', 'minQty': '0.00001000', 'maxQty': '9000.00000000', 'stepSize': '0.00001000'}, {'filterType': 'MIN_NOTIONAL', 'minNotional': '10.00000000', 'applyToMarket': True, 'avgPriceMins': '5'}, {'filterType': 'ICEBERG_PARTS', 'limit': '10'}, {'filterType': 'MARKET_LOT_SIZE', 'minQty': '0.00000000', 'maxQty': '144.59134726', 'stepSize': '0.00000000'}, {'filterType': 'TRAILING_DELTA', 'minTrailingAboveDelta': '10', 'maxTrailingAboveDelta': '2000', 'minTrailingBelowDelta': '10', 'maxTrailingBelowDelta': '2000'}, {'filterType': 'PERCENT_PRICE_BY_SIDE', 'bidMultiplierUp': '5', 'bidMultiplierDown': '0.2', 'askMultiplierUp': '5', 'askMultiplierDown': '0.2', 'avgPriceMins': '5'}, {'filterType': 'MAX_NUM_ORDERS', 'maxNumOrders': '200'}, {'filterType': 'MAX_NUM_ALGO_ORDERS', 'maxNumAlgoOrders': '5'}]
[{'filterType': 'PRICE_FILTER', 'minPrice': '0.01000000', 'maxPr

In [19]:
signal

,BTC/BUSD,ETH/BUSD,BNB/BUSD
timestamp,,,
2022-12-23 23:00:00,nan,sell,sell
2022-12-24 00:00:00,nan,nan,nan
2022-12-24 01:00:00,nan,buy,buy
2022-12-24 02:00:00,nan,nan,nan
2022-12-24 03:00:00,nan,nan,nan
...,...,...,...
2023-01-13 14:00:00,buy,buy,buy
2023-01-13 15:00:00,nan,nan,nan
2023-01-13 16:00:00,sell,sell,sell


In [66]:
print('start')
while True: 
    coin_dict = {}


    for ticker in ticker_list:
        bars = ex.fetch_ohlcv(ticker, timeframe='1m', limit=5)
        df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.set_index('timestamp')
        coin_dict[ticker] = df

        df_coin = pd.concat(coin_dict)

        x = strategy(df_coin)
        print(x)
#        print(x[-1])

        time.sleep(5)  


start
                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:38:00  19295.97  19302.20  19291.71  19300.40   
         2023-01-13 19:39:00  19300.41  19313.73  19299.35  19307.10   
         2023-01-13 19:40:00  19308.07  19316.74  19302.20  19315.76   
         2023-01-13 19:41:00  19315.91  19334.00  19313.02  19324.54   
         2023-01-13 19:42:00  19324.50  19333.56  19323.31  19329.95   

                                 volume           ema           sma  trend  \
         timestamp                                                           
BTC/BUSD 2023-01-13 19:38:00   67.49489           NaN           NaN  False   
         2023-01-13 19:39:00   91.75117           NaN           NaN  False   
         2023-01-13 19:40:00   67.97206  19307.753333  19307.753333  False   
         2023-01-13 19:41:00  158.97307  19316.146667  19315.800000   True   
         2023-01-13 1

                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:38:00  19295.97  19302.20  19291.71  19300.40   
         2023-01-13 19:39:00  19300.41  19313.73  19299.35  19307.10   
         2023-01-13 19:40:00  19308.07  19316.74  19302.20  19315.76   
         2023-01-13 19:41:00  19315.91  19334.00  19313.02  19324.54   
         2023-01-13 19:42:00  19324.50  19333.56  19322.82  19331.13   
ETH/BUSD 2023-01-13 19:38:00   1416.47   1416.62   1416.32   1416.61   
         2023-01-13 19:39:00   1416.61   1417.21   1416.61   1416.88   
         2023-01-13 19:40:00   1416.88   1417.38   1416.88   1417.38   
         2023-01-13 19:41:00   1417.38   1417.90   1417.31   1417.38   
         2023-01-13 19:42:00   1417.38   1417.98   1417.37   1417.63   
BNB/BUSD 2023-01-13 19:38:00    287.60    287.70    287.50    287.70   
         2023-01-13 19:39:00    287.60    287.80    287.60    28

                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:39:00  19300.41  19313.73  19299.35  19307.10   
         2023-01-13 19:40:00  19308.07  19316.74  19302.20  19315.76   
         2023-01-13 19:41:00  19315.91  19334.00  19313.02  19324.54   
         2023-01-13 19:42:00  19324.50  19333.56  19322.82  19325.80   
         2023-01-13 19:43:00  19325.87  19335.95  19325.87  19333.39   

                                 volume        ema           sma  trend signal  
         timestamp                                                              
BTC/BUSD 2023-01-13 19:39:00   91.75117        NaN           NaN  False    nan  
         2023-01-13 19:40:00   67.97206        NaN           NaN  False    nan  
         2023-01-13 19:41:00  158.97307  19315.800  19315.800000  False    nan  
         2023-01-13 19:42:00  192.84660  19320.800  19322.033333  False    nan  
         

                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:39:00  19300.41  19313.73  19299.35  19307.10   
         2023-01-13 19:40:00  19308.07  19316.74  19302.20  19315.76   
         2023-01-13 19:41:00  19315.91  19334.00  19313.02  19324.54   
         2023-01-13 19:42:00  19324.50  19333.56  19322.82  19325.80   
         2023-01-13 19:43:00  19325.87  19335.95  19325.87  19332.16   
ETH/BUSD 2023-01-13 19:39:00   1416.61   1417.21   1416.61   1416.88   
         2023-01-13 19:40:00   1416.88   1417.38   1416.88   1417.38   
         2023-01-13 19:41:00   1417.38   1417.90   1417.31   1417.38   
         2023-01-13 19:42:00   1417.38   1417.98   1417.37   1417.63   
         2023-01-13 19:43:00   1417.63   1417.86   1417.62   1417.68   
BNB/BUSD 2023-01-13 19:39:00    287.60    287.80    287.60    287.70   
         2023-01-13 19:40:00    287.80    287.80    287.70    28

                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:40:00  19308.07  19316.74  19302.20  19315.76   
         2023-01-13 19:41:00  19315.91  19334.00  19313.02  19324.54   
         2023-01-13 19:42:00  19324.50  19333.56  19322.82  19325.80   
         2023-01-13 19:43:00  19325.87  19335.95  19325.87  19329.90   
         2023-01-13 19:44:00  19329.90  19329.90  19329.90  19329.90   

                                 volume           ema           sma  trend  \
         timestamp                                                           
BTC/BUSD 2023-01-13 19:40:00   67.97206           NaN           NaN  False   
         2023-01-13 19:41:00  158.97307           NaN           NaN  False   
         2023-01-13 19:42:00  192.84660  19322.033333  19322.033333   True   
         2023-01-13 19:43:00  119.01559  19325.966667  19326.746667  False   
         2023-01-13 19:44:0

...

                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:44:00  19330.25  19336.00  19312.86  19322.62   
         2023-01-13 19:45:00  19322.84  19336.15  19319.21  19326.69   
         2023-01-13 19:46:00  19328.07  19340.98  19321.60  19340.16   
         2023-01-13 19:47:00  19339.31  19348.79  19336.11  19343.01   
         2023-01-13 19:48:00  19342.78  19361.91  19340.42  19356.99   

                                 volume           ema           sma  trend  \
         timestamp                                                           
BTC/BUSD 2023-01-13 19:44:00  133.14372           NaN           NaN  False   
         2023-01-13 19:45:00  153.82460           NaN           NaN  False   
         2023-01-13 19:46:00  141.16579  19329.823333  19329.823333  False   
         2023-01-13 19:47:00  140.37371  19336.416667  19336.620000  False   
         2023-01-13 19:48:0

                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:44:00  19330.25  19336.00  19312.86  19322.62   
         2023-01-13 19:45:00  19322.84  19336.15  19319.21  19326.69   
         2023-01-13 19:46:00  19328.07  19340.98  19321.60  19340.16   
         2023-01-13 19:47:00  19339.31  19348.79  19336.11  19343.01   
         2023-01-13 19:48:00  19342.78  19370.08  19340.42  19359.81   
ETH/BUSD 2023-01-13 19:45:00   1416.93   1417.89   1416.75   1417.53   
         2023-01-13 19:46:00   1417.63   1418.68   1417.46   1418.64   
         2023-01-13 19:47:00   1418.64   1418.96   1418.56   1418.96   
         2023-01-13 19:48:00   1418.96   1420.40   1418.96   1419.81   
         2023-01-13 19:49:00   1419.81   1419.81   1419.81   1419.81   
BNB/BUSD 2023-01-13 19:45:00    287.80    288.00    287.80    288.00   
         2023-01-13 19:46:00    288.00    288.10    288.00    28

                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:45:00  19322.84  19336.15  19319.21  19326.69   
         2023-01-13 19:46:00  19328.07  19340.98  19321.60  19340.16   
         2023-01-13 19:47:00  19339.31  19348.79  19336.11  19343.01   
         2023-01-13 19:48:00  19342.78  19370.08  19340.42  19357.47   
         2023-01-13 19:49:00  19356.93  19373.99  19355.00  19364.23   

                                 volume         ema           sma  trend  \
         timestamp                                                         
BTC/BUSD 2023-01-13 19:45:00  153.82460         NaN           NaN  False   
         2023-01-13 19:46:00  141.16579         NaN           NaN  False   
         2023-01-13 19:47:00  140.37371  19336.6200  19336.620000  False   
         2023-01-13 19:48:00  340.07040  19347.0450  19346.880000   True   
         2023-01-13 19:49:00   87.95731

                                  open      high       low     close  \
         timestamp                                                     
BTC/BUSD 2023-01-13 19:45:00  19322.84  19336.15  19319.21  19326.69   
         2023-01-13 19:46:00  19328.07  19340.98  19321.60  19340.16   
         2023-01-13 19:47:00  19339.31  19348.79  19336.11  19343.01   
         2023-01-13 19:48:00  19342.78  19370.08  19340.42  19357.47   
         2023-01-13 19:49:00  19356.93  19373.99  19355.00  19369.55   
ETH/BUSD 2023-01-13 19:45:00   1416.93   1417.89   1416.75   1417.53   
         2023-01-13 19:46:00   1417.63   1418.68   1417.46   1418.64   
         2023-01-13 19:47:00   1418.64   1418.96   1418.56   1418.96   
         2023-01-13 19:48:00   1418.96   1420.40   1418.96   1419.81   
         2023-01-13 19:49:00   1419.82   1420.69   1419.81   1420.56   
BNB/BUSD 2023-01-13 19:45:00    287.80    288.00    287.80    288.00   
         2023-01-13 19:46:00    288.00    288.10    288.00    28

KeyboardInterrupt: 

In [ ]:
# import schedule
# import time

# # print('Start Bot ') 

# schedule.every(5).minutes.at(":01").do(run_bot)

# while True:

#     schedule.run_pending()
#     time.sleep(1)
    